# ASOS Lookup Function

A user can pass either the ASOS airport code or the “city” and the script/function identifes and returns the name of the HDP station. 
E.g., “I need to find KSAC weather station” or “I need to find the Sacramento Executive Airport weather station” 

Script then identifies KSAC as “ASOSAWOS_XXXXXXXX”

Reasoning: Looking up the specific HDP stations from the attached spreadsheet was not straightforward or easy; users may very likely come to HDP knowing specifically which station they are looking for, and it’s not obvious which station is which.


In [1]:
# imports
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import os
from shapely.geometry import Point
import sys  # Used for progress bar
import datetime
import sys
import os
from io import BytesIO

plt.style.use("tableau-colorblind10")

# Set AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client("s3")  # for lower-level processes

# Set relative paths to other folders and objects in repository.
BUCKET_NAME = "wecc-historical-wx"
QAQC_DIR = "3_qaqc_wx"
MERGE_DIR = "4_merge_wx"
stations_csv_path = f"s3://{BUCKET_NAME}/{MERGE_DIR}/all_network_stationlist_merge.csv"

## Development

grab the station codes from the pull station list version and append as a new column into the merge station list.

In [5]:
# load merge station list
merge_list = pd.read_csv(
    f"s3://wecc-historical-wx/{MERGE_DIR}/ASOSAWOS/stationlist_ASOSAWOS_merge.csv"
)
merge_list.head(1)

,ERA-ID,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),...,sfcWind_dir_nobs,rsds,rsds_nobs,total_nobs,QAQC,Time_QAQC,Errors_QAQC,merged,Time_Merge,Errors_Merge
0,ASOSAWOS_72681024131,726810,24131,BOISE AIR TERMINAL/GOWEN FD AIRPORT,US,ID,KBOI,43.567,-116.241,860.5,...,379116,Y,87216,470754,Y,2025-05-01 19:48:20+00:00,NaN,Y,2025-06-12 21:37:35+00:00,NaN


In [12]:
merge_list[['ERA-ID','STATION NAME', 'STATE', 'ICAO','LAT', 'LON','NAME', 'ALTNAME', 'COUNTRY','ST', 'COUNTY', 'ELEV', 'UTC', 'STNTYPE']]

,ERA-ID,STATION NAME,STATE,ICAO,LAT,LON,NAME,ALTNAME,COUNTRY,ST,COUNTY,ELEV,UTC,STNTYPE
0,ASOSAWOS_72681024131,BOISE AIR TERMINAL/GOWEN FD AIRPORT,ID,KBOI,43.567,-116.241,BOISE AIR TERMINAL,BOISE AIR TERMINAL,UNITED STATES,ID,ADA,860.4504,-7.0,"ASOS,COOP,PLCD"
1,ASOSAWOS_72677024033,BILLINGS LOGAN INTERNATIONAL AIRPORT,MT,KBIL,45.807,-108.546,BILLINGS INTL AP,BILLINGS INTL AP,UNITED STATES,MT,YELLOWSTONE,1094.2320,-7.0,"ASOS,COOP,PLCD"
2,ASOSAWOS_72698024229,PORTLAND INTERNATIONAL AIRPORT,OR,KPDX,45.596,-122.609,PORTLAND INTL AP,PORTLAND INTL AP,UNITED STATES,OR,MULTNOMAH,6.7056,-8.0,"ASOS,COOP,PLCD"
3,ASOSAWOS_72569024089,NATRONA COUNTY INTERNATIONAL AP,WY,KCPR,42.898,-106.474,CASPER-NATRONA CO AP,CASPER-NATRONA CO AP,UNITED STATES,WY,NATRONA,1621.2312,-7.0,"AIRWAYS,ASOS,COOP,PLCD,WXSVC"
4,ASOSAWOS_72297793184,J. WAYNE APT-ORANGE CO APT,CA,KSNA,33.680,-117.867,SANTA ANA JOHN WAYNE AP,SANTA ANA JOHN WAYNE AP,UNITED STATES,CA,ORANGE,13.1064,-8.0,"AIRWAYS,ASOS"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,ASOSAWOS_A0705300346,TRINITY CENTER AIRPORT,CA,KO86,40.983,-122.694,TRINITY CENTER AP,NaN,UNITED STATES,CA,TRINITY,728.1672,-8.0,AWOS
471,ASOSAWOS_A0704900320,PETALUMA MUNICIPAL AIRPORT,CA,KO69,38.250,-122.600,PETALUMA MUNI AP,NaN,UNITED STATES,CA,SONOMA,27.1272,-8.0,AWOS
472,ASOSAWOS_A0685400115,BIG BEAR CITY AIRPORT,CA,KL35,34.264,-116.854,BIG BEAR CITY AP,NaN,UNITED STATES,CA,SAN BERNARDINO,2057.0952,-8.0,AWOS
473,ASOSAWOS_A0001823162,DELTA MUNICIPAL AIRPORT,UT,KDTA,39.383,-112.517,DELTA FAA AP,NaN,UNITED STATES,UT,MILLARD,1450.5432,-7.0,AWOS


In [32]:
merge_list['NAME']

0            BOISE AIR TERMINAL
1              BILLINGS INTL AP
2              PORTLAND INTL AP
3          CASPER-NATRONA CO AP
4       SANTA ANA JOHN WAYNE AP
                 ...           
470           TRINITY CENTER AP
471            PETALUMA MUNI AP
472            BIG BEAR CITY AP
473                DELTA FAA AP
474    SALT LAKE CITY MUNI AP 2
Name: NAME, Length: 475, dtype: object

In [33]:
code_dict = pd.Series(
    merge_list["ERA-ID"].values, index=merge_list["ICAO"]
).to_dict()

In [91]:
city_dict = pd.Series(merge_list["ERA-ID"].values, index=merge_list["STATION NAME"]).to_dict()

In [95]:
[value for key, value in city_dict.items() if city in key]

['ASOSAWOS_72681024131']

In [ ]:

hdp_station = hdp_station[0]

In [105]:
def asos_station_lookup(code: str | None = None, city: str | None = None) -> None:
    """
    Function that returns the HDP station ID for an input of either the
    1. four digit code or (e.g. KSAC)
    2. city (e.g. Sacramento)

    of the airport associated with the station.

    This functon is specific to the 16 stations of interest shared by project partners.

    Paramters
    ---------
    code : str
        ASOS airport code (e.g. KSAC weather station)
    city : str
        ASOS aiport city (e.g. Sacramento Executive Airport weather station)

    Returns
    -------
    None

    """
    # Define dictionaries matching HDP station IDs to airport codes and cities
    ## this was developed beforehand matching HDP ASOSAWOS station locations with those in a list of stations provided by partners

    merge_list = pd.read_csv(
        f"s3://wecc-historical-wx/{MERGE_DIR}/ASOSAWOS/stationlist_ASOSAWOS_merge.csv"
    )
    code_dict = pd.Series(
        merge_list["ERA-ID"].values, index=merge_list["ICAO"]
    ).to_dict()
    city_dict = pd.Series(merge_list["ERA-ID"].values, index=merge_list["STATION NAME"]).to_dict()

    # If user inputs an airport code
    if code:
        if code in code_dict:
            hdp_station = code_dict[code]
            print(
                f"The HDP station name for input airport code {code} is {hdp_station}"
            )
        else:
            print(f"Input code '{code}' not in station dictionary.")
    # If user inputs a city
    elif city:
        # this catches cases in which the user inputs the entire airport name
        hdp_station = [val for key, val in city_dict.items() if city in key]
        if len(hdp_station)==1:
            # now pull the ID out from the list that is returned above
            hdp_station = hdp_station[0]
            print(
                f"The HDP station name for input airport city '{city}' is {hdp_station}"
            )
        elif len(hdp_station) > 1:
            print(f"There are multiple stations associated with '{city}': {hdp_station}")
        else:
            print(f"Input city '{city}' not in station dictionary.")
    else:
        print(
            "Please input either an airport code (e.g. code='KSAC') or city (e.g. city='Sacramento')."
        )

    return None

In [108]:
asos_station_lookup(code=None, city='nan')

Input city 'nan' not in station dictionary.


## Scraps

I matched up ASOSAWOS stations from the merge station list with the lat lons of the stations in the list that Mithra shared.

In [ ]:
# load station list
station_list = pd.read_csv(stations_csv_path)
station_list = station_list[station_list['network']=='ASOSAWOS']

In [45]:
# convert to geodataframe
asos_stns = gpd.GeoDataFrame(
    station_list,
    geometry=gpd.points_from_xy(
        station_list.longitude, station_list.latitude, crs="EPSG:4326"
    ),
)

In [ ]:
# load in list that Mithra shared (stored in separate repo)
lookup_table = pd.read_csv("../../skillshare_demo/lookup_table.csv")

In [ ]:
# convert it to a geodataframe
lookup_table = gpd.GeoDataFrame(
    lookup_table,
    geometry=gpd.points_from_xy(
        lookup_table.longitude, lookup_table.latitude, crs="EPSG:4326"
    ),
)

In [ ]:
# and filter for ease of use later
lookup_table = lookup_table[
    ["Airport Code", "City", "latitude", "longitude",'geometry']
]

In [ ]:
# establish buffers around the point geometries, as they will not be exact matches

# define the error margin (buffer distance)
error_margin = 0.1  # Adjust this value as needed

# apply buffer to account for the error margin
lookup_table_buffered = lookup_table.copy()
lookup_table_buffered["geometry"] = lookup_table.geometry.buffer(error_margin)

asos_stns_buffered = asos_stns.copy()
asos_stns_buffered["geometry"] = asos_stns.geometry.buffer(error_margin)

/tmp/ipykernel_31410/1684058703.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lookup_table_buffered["geometry"] = lookup_table.geometry.buffer(error_margin)
/tmp/ipykernel_31410/1684058703.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  asos_stns_buffered["geometry"] = asos_stns.geometry.buffer(error_margin)


In [ ]:
# perform the overlay operation (e.g., intersection)
matchup_buffered = gpd.overlay(asos_stns_buffered, lookup_table_buffered, how="intersection")

In [ ]:
# clean things up 
matchup_buffered = matchup_buffered[["era-id", "longitude_1", "latitude_1","longitude_2", "latitude_2",'Airport Code','City']]

Now go through each city in the list (which I added manually before starting this process) and check which HDP station is a match. I added the station ID to a copy of the list in my drive along the way.

In [102]:
matchup_buffered[matchup_buffered["City"] == "San Jose"]

,era-id,longitude_1,latitude_1,longitude_2,latitude_2,Airport Code,City
19,ASOSAWOS_72494523293,-121.924,37.359,-121.82,37.33,KRHV,San Jose


I then downloaded a cleaned version of the table and load it in here (again, stored in a separate repo).

In [ ]:
matchup_table = pd.read_csv("../../skillshare_demo/final_lookup_table.csv")

And generated two dictionaries from it - one for airport codes, another for airport cities

In [123]:
code_dict = pd.Series(matchup_table["HDP station"].values, index=matchup_table["Airport Code"]).to_dict()

In [124]:
city_dict = pd.Series(
    matchup_table["HDP station"].values, index=matchup_table["City"]
).to_dict()

The dictionaries are generated manually in the function itself.